# Data Integration: AQI vs. Median Household Income

### Objective
This notebook documents the systematic process of joining air quality data with socio-economic indicators. By merging these datasets, we can analyze the relationship between local economic status and environmental health outcomes across US counties.

### Key Steps:
1. **Normalization**: Standardizing county and state names across disparate data sources.
2. **Merge Operation**: Performing a high-precision inner join.
3. **Validation**: Assessing the coverage and quality of the integrated dataset.

In [1]:
import pandas as pd
import numpy as np

# File Paths
AQI_DATA_PATH = 'aqi-datasets/ml_target_dataset.csv'
INCOME_DATA_PATH = 'median-household-income-2024-datasets/cleaned_median_household_income.csv'
OUTPUT_PATH = 'aqi_income_joined.csv'

def load_data():
    aqi_df = pd.read_csv(AQI_DATA_PATH)
    income_df = pd.read_csv(INCOME_DATA_PATH)
    return aqi_df, income_df

aqi_df, income_df = load_data()

print(f"AQI Dataset: {aqi_df.shape[0]} rows")
print(f"Income Dataset: {income_df.shape[0]} rows")
aqi_df.head(3)

AQI Dataset: 978 rows
Income Dataset: 3221 rows


,State,County,Year,median_aqi,sample_weight
0,Alabama,Baldwin,2025,42,1.0
1,Alabama,Clay,2025,32,1.0
2,Alabama,DeKalb,2025,42,1.0


## 1. Data Normalization

The household income dataset formats its area as `"County Name, State Name"` (e.g., `"Baldwin County, Alabama"`), while the AQI dataset keeps these in separate columns. 

To ensure a successful join, we must:
- Split the income dataset's location into separate columns.
- Remove descriptive suffixes (like " County", " Borough", " Parish") that are inconsistent between datasets.
- Standardize capitalization and whitespace.

In [2]:
def normalize_income_data(df):
    # Split 'County_Area' into County and State
    split_data = df['County_Area'].str.split(', ', expand=True)
    df['County'] = split_data[0]
    df['State'] = split_data[1]
    
    # List of suffixes to remove to match the AQI dataset format
    suffixes = [
        ' County', ' Borough', ' Census Area', ' Municipality', 
        ' City and Borough', ' Parish', ' City', ' City and County'
    ]
    
    # Clean County Names
    for suffix in suffixes:
        df['County'] = df['County'].str.replace(suffix, '', regex=False)
    
    # Strip whitespace and standardize strings
    df['County'] = df['County'].str.strip()
    df['State'] = df['State'].str.strip()
    
    return df

income_df_cleaned = normalize_income_data(income_df.copy())
print("Normalized Income Data Sample:")
income_df_cleaned[['County', 'State', 'Median_Household_Income']].head(5)

Normalized Income Data Sample:


,County,State,Median_Household_Income
0,Autauga,Alabama,72481.0
1,Baldwin,Alabama,78775.0
2,Barbour,Alabama,46042.0
3,Bibb,Alabama,52541.0
4,Blount,Alabama,64190.0


## 2. Inner Join

We perform an inner join on `State` and `County`. This ensures we only analyze counties where both environmental and economic data are verified and available.

In [3]:
# Perform Merge
joined_df = pd.merge(
    aqi_df, 
    income_df_cleaned[['County', 'State', 'Median_Household_Income']], 
    on=['State', 'County'], 
    how='inner'
)

# Calculate success rate
aqi_count = aqi_df.shape[0]
joined_count = joined_df.shape[0]
success_rate = (joined_count / aqi_count) * 100

print(f"Original AQI Counties: {aqi_count}")
print(f"Matched Counties: {joined_count}")
print(f"Join Success Rate: {success_rate:.2f}%")

joined_df.head(10)

Original AQI Counties: 978
Matched Counties: 940
Join Success Rate: 96.11%


,State,County,Year,median_aqi,sample_weight,Median_Household_Income
0,Alabama,Baldwin,2025,42,1.000000,78775.0
1,Alabama,Clay,2025,32,1.000000,55250.0
2,Alabama,DeKalb,2025,42,1.000000,51204.0
3,Alabama,Elmore,2025,32,0.983333,78243.0
4,Alabama,Etowah,2025,45,1.000000,54563.0
5,Alabama,Jefferson,2025,53,1.000000,66388.0
6,Alabama,Lawrence,2025,23,0.105556,66071.0
7,Alabama,Madison,2025,42,1.000000,86499.0
8,Alabama,Mobile,2025,43,1.000000,58880.0
9,Alabama,Montgomery,2025,44,1.000000,59386.0


## 3. Export Results

Finally, we export the clean, integrated dataset for downstream machine learning and visualization.

In [4]:
joined_df.to_csv(OUTPUT_PATH, index=False)
print(f"Successfully exported joined data to {OUTPUT_PATH}")

Successfully exported joined data to aqi_income_joined.csv
